# <div style='padding:25px;background-color:maroon;color:white;border-radius:4px;font-size:100%;text-align: center'>Banking Data Analysis<br></div>

# <div style='padding:7px;background-color:Maroon;color:white;border-radius:2px;font-size:100%;text-align: center'>Regression Modeling<br></div>

## <span style="color:Aqua;">Importing libraries from Python</span>

In [34]:
import pandas as pd
pd.options.display.max_columns = 50
pd.set_option("display.precision", 4)
pd.set_option('display.float_format', '{:.4f}'.format)

import numpy as np

import seaborn as sns
sns.set_theme(context='notebook', style='white', palette='dark', font='sans-serif', font_scale=1, color_codes=True, rc=None)
sns.set_style({'axes.facecolor': 'white','axes.edgecolor': 'white'})

import matplotlib.pyplot as plt
plt.rcParams.update({
    'text.color': 'black',  # Change font color
    'axes.labelcolor': 'black',  # Change axis label color
    'xtick.color': 'black',  # Change x-tick color
    'ytick.color': 'black',  # Change y-tick color
    'font.weight': 'bold',  # Change font weight
    'axes.titleweight': 'bold',  # Change title weight
    'axes.labelweight': 'bold',# Change label weight
})

from sklearn.preprocessing import StandardScaler, RobustScaler,MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score

from sklearn.linear_model import LinearRegression,Ridge,Lasso,ElasticNet
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import AdaBoostRegressor,GradientBoostingRegressor, RandomForestRegressor, ExtraTreesRegressor

from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, median_absolute_error, explained_variance_score,root_mean_squared_error
import xgboost as xg

import pickle
import time



## <span style="color:Aqua;">Reading model dataset:</span>

In [35]:
df = pd.read_feather('clustered.feather')

In [36]:
df.sample(3)

,num_bank_accounts,num_credit_card,num_of_loan,delay_from_due_date,num_of_delayed_payment,num_credit_inquiries,credit_mix,payment_of_min_amount,payment_behaviour,credit_score,Mortgage Loan,Home Equity Loan,Credit-Builder Loan,Payday Loan,Personal Loan,Debt Consolidation Loan,Student Loan,Auto Loan,credit_inquiry_per_loan,age_group,disposable_income,age_boxcox,annual_income_boxcox,monthly_inhand_salary_boxcox,interest_rate_boxcox,outstanding_debt_square,credit_utilization_ratio_log,credit_history_age_boxcox,changed_credit_limit_cube,total_emi_per_month_square,amount_invested_monthly_boxcox,monthly_balance_boxcox,credit_limit_change_rate_exp,debt_income_ratio_log,loan_income_ratio_month_boxcox,credit_utilization_per_account_boxcox,reliable_payment_boxcox,delayed_payment_ratio_boxcox,investment_to_income_ratio_square,total_credit_exposure_boxcox,credit_utilization_to_income_ratio_boxcox,savings_rate_rec,income_stability_boxcox,total_liability_boxcox,debt_paytime_log,Cluster
710,7,3,4,7,13,6.0000,1,0,1,1,0,1,1,0,1,0,1,0,2.0000,0,2445.6058,7.5176,13.2825,11.1066,4.6482,30.3275,3.6109,191.3813,125,8.5576,4.7451,18.6595,0.4259,8.1267,1.8489,0.9046,1.4888,1.1063,0.0838,23.2928,-8.6007,142.3971,0.1167,24.0136,2.5169,0
95174,6,10,9,52,24,7.0000,0,2,1,0,1,0,0,1,1,1,1,1,1.0000,1,6157.5627,8.1014,15.0849,13.0279,9.1806,43.3170,3.5835,135.8583,1,19.0964,8.4170,20.3121,0.3336,8.2692,1.6788,0.9364,2.9557,1.0010,0.0942,29.5372,-10.4864,112.6286,0.0029,31.7957,1.6354,1
20523,5,7,0,11,16,7.0000,1,2,2,1,0,0,0,0,0,0,0,0,7.0000,2,1774.8492,10.4710,12.8001,10.4376,6.7454,13.7764,3.6889,189.9657,87,0.0000,5.3946,19.5849,0.4563,7.8337,2.2968,1.0042,1.7792,3.5947,0.1129,13.9244,-7.9920,78.4731,0.0547,13.1149,5.2459,0


In [51]:
continuous_columns = [
      'disposable_income', 'age_boxcox', 
    'annual_income_boxcox', 'monthly_inhand_salary_boxcox', 'interest_rate_boxcox', 
    'outstanding_debt_square', 'credit_utilization_ratio_log', 'credit_history_age_boxcox', 
    'total_emi_per_month_square',  'monthly_balance_boxcox', 
    'credit_limit_change_rate_exp', 'debt_income_ratio_log', 'loan_income_ratio_month_boxcox', 
    'credit_utilization_per_account_boxcox', 'reliable_payment_boxcox', 'delayed_payment_ratio_boxcox', 
    'investment_to_income_ratio_square', 'total_credit_exposure_boxcox', 
    'credit_utilization_to_income_ratio_boxcox', 'savings_rate_rec', 'income_stability_boxcox', 
    'total_liability_boxcox', 'debt_paytime_log','changed_credit_limit_cube'
]

Ordinal_columns = ['num_bank_accounts', 'num_credit_card', 'num_of_loan', 'delay_from_due_date', 'num_credit_inquiries','num_of_delayed_payment', 'credit_mix', 'payment_of_min_amount', 'payment_behaviour', 
      'age_group', 'Cluster' ,'credit_inquiry_per_loan','credit_score'
]

print(len(continuous_columns) + len(Ordinal_columns))

37


In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 46 columns):
 #   Column                                     Non-Null Count   Dtype  
---  ------                                     --------------   -----  
 0   num_bank_accounts                          100000 non-null  int8   
 1   num_credit_card                            100000 non-null  int8   
 2   num_of_loan                                100000 non-null  int8   
 3   delay_from_due_date                        100000 non-null  int8   
 4   num_of_delayed_payment                     100000 non-null  int8   
 5   num_credit_inquiries                       100000 non-null  float64
 6   credit_mix                                 100000 non-null  int8   
 7   payment_of_min_amount                      100000 non-null  int8   
 8   payment_behaviour                          100000 non-null  int8   
 9   credit_score                               100000 non-null  int8   
 10  Mortgage 

## <span style="color:Aqua;">Preparing the data afor modeling:</span>

In [53]:
x = df.drop('amount_invested_monthly_boxcox',axis=1)
y = df[['amount_invested_monthly_boxcox']]

In [54]:
x.describe().T

,count,mean,std,min,25%,50%,75%,max
num_bank_accounts,100000.0000,5.3688,2.5933,0.0000,3.0000,5.0000,7.0000,11.0000
num_credit_card,100000.0000,5.5336,2.0671,0.0000,4.0000,5.0000,7.0000,11.0000
num_of_loan,100000.0000,3.5329,2.4464,0.0000,2.0000,3.0000,5.0000,9.0000
delay_from_due_date,100000.0000,21.0814,14.8046,0.0000,10.0000,18.0000,28.0000,62.0000
num_of_delayed_payment,100000.0000,13.3131,6.2372,0.0000,9.0000,14.0000,18.0000,25.0000
num_credit_inquiries,100000.0000,5.7983,3.8678,0.0000,3.0000,5.0000,8.0000,17.0000
credit_mix,100000.0000,1.0662,0.7329,0.0000,1.0000,1.0000,2.0000,2.0000
payment_of_min_amount,100000.0000,1.1666,0.9231,0.0000,0.0000,2.0000,2.0000,2.0000
payment_behaviour,100000.0000,2.2379,1.8613,0.0000,0.0000,2.0000,4.0000,5.0000
credit_score,100000.0000,0.8883,0.6751,0.0000,0.0000,1.0000,1.0000,2.0000


In [55]:
y.describe()

,amount_invested_monthly_boxcox
count,100000.0000
mean,7.6677
std,2.4281
min,1.1662
25%,5.9981
50%,7.5257
75%,9.2194
max,14.0514


In [56]:
y.value_counts()

amount_invested_monthly_boxcox
1.1662                            1920
14.0514                            888
3.4252                               8
8.6048                               8
8.5990                               8
                                  ... 
6.5951                               8
6.5958                               8
6.5960                               8
6.5963                               8
7.5628                               8
Name: count, Length: 12151, dtype: int64

In [59]:
original_features=['num_bank_accounts', 'num_credit_card', 'num_of_loan', 'delay_from_due_date','num_of_delayed_payment',
     'num_credit_inquiries', 'credit_mix','payment_of_min_amount', 'payment_behaviour', 'age_boxcox',
     'total_emi_per_month_square','amount_invested_monthly_boxcox','annual_income_boxcox',]

all_features = ['num_bank_accounts', 'num_credit_card', 'num_of_loan', 'delay_from_due_date',
    'num_of_delayed_payment', 'num_credit_inquiries', 'credit_mix','payment_of_min_amount', 'payment_behaviour',
    'reliable_payment_boxcox', 'delayed_payment_ratio_boxcox','total_credit_exposure_boxcox',
    'credit_utilization_to_income_ratio_boxcox', 'savings_rate_rec','debt_paytime_log','interest_rate_boxcox',
    'age_boxcox','total_emi_per_month_square','amount_invested_monthly_boxcox','annual_income_boxcox',
             'income_stability_boxcox','total_liability_boxcox','investment_to_income_ratio_square',
             'outstanding_debt_square','credit_inquiry_per_loan','changed_credit_limit_cube']

revised_features = ['num_bank_accounts','num_credit_card','num_of_loan','num_of_delayed_payment', 'delay_from_due_date',
                    'num_credit_inquiries', 'credit_mix','payment_of_min_amount', 'payment_behaviour',
                    'age_boxcox','total_emi_per_month_square','amount_invested_monthly_boxcox',
                    'annual_income_boxcox','total_liability_boxcox','outstanding_debt_square','savings_rate_rec']

imp_fea = ['credit_mix','outstanding_debt_square','credit_utilization_ratio_log','interest_rate_boxcox','payment_of_min_amount',
    'total_liability_boxcox','num_credit_card','num_bank_accounts','Cluster','total_credit_exposure_boxcox',
    'num_credit_inquiries','num_of_delayed_payment','delay_from_due_date','payment_behaviour','credit_limit_change_rate_exp',
    'loan_income_ratio_month_boxcox','disposable_income','annual_income_boxcox','total_emi_per_month_square',
    'monthly_inhand_salary_boxcox','debt_income_ratio_log','amount_invested_monthly_boxcox','savings_rate_rec',
    'income_stability_boxcox','age_boxcox','changed_credit_limit_cube']


## <span style="color:Aqua;">Splitting data for train_test and resampling imbalanced data classess:</span>

In [60]:
x_train, x_test,y_train, y_test = train_test_split(x,y,test_size=.3,random_state=42)

In [61]:
print(x_train.shape,'----', y_train.shape)
print(x_test.shape,'----', y_test.shape)

(70000, 45) ---- (70000, 1)
(30000, 45) ---- (30000, 1)


In [62]:
x_test.sample(2)

,num_bank_accounts,num_credit_card,num_of_loan,delay_from_due_date,num_of_delayed_payment,num_credit_inquiries,credit_mix,payment_of_min_amount,payment_behaviour,credit_score,Mortgage Loan,Home Equity Loan,Credit-Builder Loan,Payday Loan,Personal Loan,Debt Consolidation Loan,Student Loan,Auto Loan,credit_inquiry_per_loan,age_group,disposable_income,age_boxcox,annual_income_boxcox,monthly_inhand_salary_boxcox,interest_rate_boxcox,outstanding_debt_square,credit_utilization_ratio_log,credit_history_age_boxcox,changed_credit_limit_cube,total_emi_per_month_square,monthly_balance_boxcox,credit_limit_change_rate_exp,debt_income_ratio_log,loan_income_ratio_month_boxcox,credit_utilization_per_account_boxcox,reliable_payment_boxcox,delayed_payment_ratio_boxcox,investment_to_income_ratio_square,total_credit_exposure_boxcox,credit_utilization_to_income_ratio_boxcox,savings_rate_rec,income_stability_boxcox,total_liability_boxcox,debt_paytime_log,Cluster
76930,5,7,0,14,6,1.0000,2,0,5,0,0,0,0,0,0,0,0,0,1.0000,4,2283.1350,15.2098,13.5060,10.9168,4.6482,36.1130,3.4340,169.0178,-40,0.0000,22.9089,0.4543,8.2611,2.3074,0.9377,0.7121,2.3715,0.1143,26.1334,-9.1250,76.5761,-0.1149,26.4053,6.8156,0
79000,5,5,3,15,9,1.0000,2,0,1,2,1,0,0,0,0,0,0,1,1.0000,4,10009.4000,15.2098,15.8959,14.0070,2.9057,32.6097,3.2958,130.3887,0,17.0300,11.4435,0.5104,9.5267,1.8586,0.8960,1.5803,0.9179,0.1147,24.3730,-11.8774,75.9975,0.0009,26.7464,1.2958,0


In [63]:
y_test.sample(2)

,amount_invested_monthly_boxcox
36325,10.2457
75827,12.5889


## <span style="color:Aqua;">Normalizing / Scaling the data:</span>

In [64]:
scaler = MinMaxScaler(feature_range=(0,1))
x_train_scaled = scaler.fit_transform(x_train[continuous_columns])
x_test_scaled = scaler.transform(x_test[continuous_columns])

In [65]:
y_train, y_test = y_train.values.ravel(), y_test.values.ravel()

In [66]:
x_train_scaled_continuous = pd.DataFrame(x_train_scaled, columns=continuous_columns)
x_test_scaled_continuous = pd.DataFrame(x_test_scaled, columns=continuous_columns)

In [67]:
x_train_scaled_df = pd.concat([x_train_scaled_continuous.reset_index(drop=True), x_train[Ordinal_columns].reset_index(drop=True)], axis=1)
x_test_scaled_df = pd.concat([x_test_scaled_continuous.reset_index(drop=True), x_test[Ordinal_columns].reset_index(drop=True)], axis=1)

In [68]:
x_train_scaled_df

,disposable_income,age_boxcox,annual_income_boxcox,monthly_inhand_salary_boxcox,interest_rate_boxcox,outstanding_debt_square,credit_utilization_ratio_log,credit_history_age_boxcox,total_emi_per_month_square,monthly_balance_boxcox,credit_limit_change_rate_exp,debt_income_ratio_log,loan_income_ratio_month_boxcox,credit_utilization_per_account_boxcox,reliable_payment_boxcox,delayed_payment_ratio_boxcox,investment_to_income_ratio_square,total_credit_exposure_boxcox,credit_utilization_to_income_ratio_boxcox,savings_rate_rec,income_stability_boxcox,total_liability_boxcox,debt_paytime_log,changed_credit_limit_cube,num_bank_accounts,num_credit_card,num_of_loan,delay_from_due_date,num_credit_inquiries,num_of_delayed_payment,credit_mix,payment_of_min_amount,payment_behaviour,age_group,Cluster,credit_inquiry_per_loan,credit_score
0,0.8432,0.2145,0.9431,0.9492,0.4786,0.2627,0.7005,0.9016,0.6741,0.7279,0.3004,0.6881,0.5800,0.7599,0.4076,0.5409,0.3548,0.3330,0.2089,0.6949,0.5506,0.3710,0.1951,0.7520,3,7,3,7,3.0000,14,2,0,2,1,0,1.0000,0
1,0.5726,0.3496,0.8083,0.8222,0.4786,0.4413,0.7565,0.9761,0.0000,1.0000,0.4611,0.8018,0.9870,0.7131,0.3919,1.0000,0.5328,0.5257,0.3380,0.4222,0.4714,0.4791,1.0000,0.0569,4,3,0,3,2.0000,14,2,0,5,2,0,2.0000,1
2,0.2725,0.6130,0.5419,0.5901,0.2063,0.4663,0.6107,0.9717,0.3891,0.5071,0.4199,0.5365,0.5416,0.8843,0.3523,0.4660,0.4120,0.5493,0.5260,0.5839,0.4596,0.5186,0.4758,0.3943,1,3,3,11,4.0000,11,2,1,3,3,0,1.0000,1
3,0.4560,0.5449,0.7376,0.7561,0.9266,0.5414,0.5465,0.2660,0.5236,0.7118,0.6127,0.6064,0.5461,0.5873,0.7210,0.5125,0.4459,0.6209,0.3454,0.5302,0.4279,0.5962,0.4387,0.8455,5,4,4,8,5.0000,16,1,2,3,2,0,1.0000,1
4,0.4875,0.7427,0.7409,0.7751,0.3398,0.4283,0.3275,0.3333,0.4325,0.7142,0.6526,0.6560,0.6308,0.3591,0.6149,0.6120,0.3968,0.5096,0.2873,0.6107,0.6049,0.4870,0.4281,0.6992,7,4,2,9,5.0000,13,1,1,3,3,0,2.0000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,0.4669,0.8047,0.7431,0.7654,0.1510,0.1404,0.6107,0.7158,0.6039,0.4468,0.3677,0.6096,0.4912,0.6095,0.3768,0.4077,0.2682,0.1798,0.3568,0.9343,0.4754,0.2517,0.0696,1.0000,5,5,3,12,1.0000,9,2,0,3,3,0,1.0000,2
69996,0.1420,0.1860,0.2187,0.3560,0.6202,0.5044,0.5465,0.4453,0.2007,0.4264,0.6210,0.2165,0.6139,0.5242,0.5889,0.6855,0.9876,0.5859,0.7776,0.1726,0.9528,0.5434,0.6587,0.0569,6,10,2,61,12.0000,16,1,2,0,1,1,4.0000,0
69997,0.4046,0.6352,0.6777,0.7208,0.7181,0.3425,0.6714,0.8530,0.5949,0.4473,0.3087,0.5327,0.4604,0.7530,0.3611,0.3750,0.4765,0.4223,0.4270,0.4879,0.6151,0.4273,0.2923,0.7520,3,6,4,8,4.0000,10,1,1,1,3,0,1.0000,1
69998,0.1709,0.9812,0.2967,0.4283,0.1510,0.2554,0.6415,0.5392,0.2816,0.3496,0.3405,0.2980,0.5435,0.8888,0.4331,0.4077,0.2570,0.3236,0.7352,0.9737,1.0000,0.2951,0.4067,0.6016,1,7,3,12,2.0000,9,2,0,2,4,0,1.0000,1


In [74]:
x_test_scaled_df

,disposable_income,age_boxcox,annual_income_boxcox,monthly_inhand_salary_boxcox,interest_rate_boxcox,outstanding_debt_square,credit_utilization_ratio_log,credit_history_age_boxcox,total_emi_per_month_square,monthly_balance_boxcox,credit_limit_change_rate_exp,debt_income_ratio_log,loan_income_ratio_month_boxcox,credit_utilization_per_account_boxcox,reliable_payment_boxcox,delayed_payment_ratio_boxcox,investment_to_income_ratio_square,total_credit_exposure_boxcox,credit_utilization_to_income_ratio_boxcox,savings_rate_rec,income_stability_boxcox,total_liability_boxcox,debt_paytime_log,changed_credit_limit_cube,num_bank_accounts,num_credit_card,num_of_loan,delay_from_due_date,num_credit_inquiries,num_of_delayed_payment,credit_mix,payment_of_min_amount,payment_behaviour,age_group,Cluster,credit_inquiry_per_loan,credit_score
0,0.2851,0.4500,0.5419,0.5994,0.4467,0.5389,0.4055,0.7340,0.0000,0.7190,0.4713,0.5657,0.9636,0.8520,0.3068,0.7293,0.4638,0.6178,0.4770,0.5048,0.5674,0.5716,1.0000,0.3943,1,5,0,9,0.0000,6,2,0,5,2,0,0.0000,2
1,0.1301,0.1270,0.2073,0.3288,0.6705,0.9764,0.7288,0.2870,0.3515,0.3283,0.8317,0.1951,0.3346,0.4864,0.7151,0.4294,0.5648,0.9825,0.8266,0.3903,0.7083,0.9306,0.6865,0.6748,8,7,6,18,9.0000,17,0,2,0,0,0,2.0000,0
2,0.7222,0.7003,0.8927,0.9007,0.4786,0.4855,0.0532,0.5178,0.6428,0.3813,0.3470,0.8086,0.5662,0.3792,0.0000,0.0000,0.3606,0.5652,0.0820,0.6823,0.5134,0.5597,0.3598,0.6016,5,2,2,3,3.0000,0,2,0,0,3,0,1.0000,2
3,0.1546,0.5678,0.3073,0.3895,0.9639,0.7116,0.5129,0.6144,0.2341,0.5028,0.5055,0.3429,0.5842,0.5107,0.4737,0.6120,0.2580,0.7715,0.6976,0.9701,0.4249,0.7243,0.7074,0.3943,6,6,2,15,11.0000,13,1,2,0,2,0,4.0000,0
4,0.5114,0.3753,0.7805,0.7936,0.5092,0.3982,0.7288,0.6539,0.6130,0.8474,0.4756,0.6794,0.5073,0.9009,0.0000,0.0000,0.4163,0.4818,0.3546,0.5766,0.4276,0.4801,0.3223,0.3333,1,2,4,1,4.0000,0,2,0,4,2,0,1.0000,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,0.3054,0.9238,0.5948,0.6218,0.6202,0.3815,0.1990,0.8128,0.0000,0.6802,0.1522,0.5433,0.9664,0.5320,0.4013,1.0000,0.3499,0.4603,0.3813,0.7057,0.3108,0.4184,0.9741,0.4959,4,7,0,8,4.0000,12,1,0,4,4,0,4.0000,1
29996,0.1900,0.7635,0.3994,0.4710,0.3776,0.5325,0.3275,0.2126,0.4147,0.3296,0.8750,0.3628,0.4032,0.5037,0.7669,0.4924,0.4060,0.6114,0.5786,0.5941,0.4785,0.5802,0.4930,0.2642,5,6,4,28,4.0000,15,1,2,4,3,0,1.0000,1
29997,0.0948,0.4742,0.1081,0.1796,0.7633,0.8599,0.3672,0.3986,0.2417,0.3920,0.7252,0.1306,0.3641,0.1831,0.6987,0.5706,0.8606,0.8920,0.8286,0.2133,0.0000,0.8415,0.7472,0.2764,10,8,5,21,6.0000,23,0,2,2,2,0,1.0000,0
29998,0.1334,0.4980,0.2126,0.3400,0.7633,0.7577,0.1040,0.0835,0.3708,0.4155,0.7575,0.1854,0.3169,0.1546,1.0000,0.4121,1.0000,0.8090,0.6840,0.1506,0.8005,0.7659,0.6095,0.7520,8,8,6,20,9.0000,16,0,2,0,2,0,2.0000,0


## <span style="color:Aqua;">Training all the model:</span>

#### <span style="color:Khaki;">Defing the function to train and evaluate the model:</span>

In [70]:
def evaluate_model(model, x_train, y_train, x_test, y_test, color_positive='\033[92m', reset_color='\033[0m'):

    print(f"\n{model}\n")

    start_time = time.time()
    model.fit(x_train, y_train)
    y_preds = model.predict(x_test)
    y_preds_train = model.predict(x_train)
    training_time = time.time() - start_time

    print(f"{color_positive}Training Time: {training_time:.2f} seconds\n")
    if isinstance(model,LinearRegression):
        print(f"{color_positive}Coefficients - {reset_color} {model.coef_} ")
    


    print(f"{color_positive}Test Mean Squared Error {reset_color} -- {mean_squared_error(y_test,y_preds):.2f}")
    print(f"{color_positive}Train Mean Squared Error {reset_color} -- {mean_squared_error(y_train,y_preds_train):.2f}\n")

    print(f"{color_positive}Test Root Mean Squared Error {reset_color} -- {root_mean_squared_error(y_test,y_preds):.2f}")
    print(f"{color_positive}Train Root Mean Squared Error {reset_color} -- {root_mean_squared_error(y_train,y_preds_train):.2f}\n")

    print(f"{color_positive}Test R2 Score {reset_color} -- {r2_score(y_test,y_preds):.2f}")
    print(f"{color_positive}Train R2 Score {reset_color} -- {r2_score(y_train,y_preds_train):.2f}\n")    

    print(f"{color_positive}Test Mean Absolute Error {reset_color} -- {mean_absolute_error(y_test,y_preds):.2f}")
    print(f"{color_positive}Train Mean Absolute Error {reset_color} -- {mean_absolute_error(y_train,y_preds_train):.2f}\n")    

    print(f"{color_positive}Test Median Absolute Error {reset_color} -- {median_absolute_error(y_test,y_preds):.2f}")
    print(f"{color_positive}Train Median Absolute Error {reset_color} -- {median_absolute_error(y_train,y_preds_train):.2f}\n")    

    print(f"{color_positive}Test Explained Var {reset_color} -- {explained_variance_score(y_test,y_preds):.2f}")
    print(f"{color_positive}Train Explained Var {reset_color} -- {explained_variance_score(y_train,y_preds_train):.2f}\n")    
    print('-'*50)

In [71]:
Linear_regressor = LinearRegression()
evaluate_model(Linear_regressor, x_train_scaled_df, y_train, x_test_scaled_df, y_test)


LinearRegression()

Training Time: 0.07 seconds

Coefficients -  [ 1.42099301e+00  1.33868942e-02  5.17537635e+00  5.06218976e+00
  1.41292125e-02  9.13742990e-02  2.61456051e-01  3.64649958e-02
 -9.68144532e-02 -6.18372058e-02  2.70282167e-02 -2.04292583e-01
 -1.81841812e-01 -2.04405453e-02  6.22760576e-02 -4.38669969e-02
  5.54377731e+00 -3.31815341e-01 -1.10171994e+00 -2.55550638e+00
  6.54040522e-01  1.91049690e-01  6.64789958e-02 -7.52811452e-03
 -1.09990736e-03 -4.27313751e-03 -1.27400466e-02 -1.14376802e-04
  2.51125500e-04 -4.55634145e-05 -7.87855822e-04 -3.97051660e-03
  3.87467465e-03 -4.43019552e-03  7.93679359e-03  2.16616319e-03
 -1.60542024e-03] 
Test Mean Squared Error  -- 0.07
Train Mean Squared Error  -- 0.07

Test Root Mean Squared Error  -- 0.27
Train Root Mean Squared Error  -- 0.27

Test R2 Score  -- 0.99
Train R2 Score  -- 0.99

Test Mean Absolute Error  -- 0.18
Train Mean Absolute Error  -- 0.18

Test Median Absolute Error  -- 0.12
Train Median Absolute Error  -

In [72]:
DT_Regressor = DecisionTreeRegressor(random_state=42)
evaluate_model(DT_Regressor, x_train_scaled_df, y_train, x_test_scaled_df, y_test)


DecisionTreeRegressor(random_state=42)

Training Time: 1.43 seconds

Test Mean Squared Error  -- 0.00
Train Mean Squared Error  -- 0.00

Test Root Mean Squared Error  -- 0.02
Train Root Mean Squared Error  -- 0.00

Test R2 Score  -- 1.00
Train R2 Score  -- 1.00

Test Mean Absolute Error  -- 0.00
Train Mean Absolute Error  -- 0.00

Test Median Absolute Error  -- 0.00
Train Median Absolute Error  -- 0.00

Test Explained Var  -- 1.00
Train Explained Var  -- 1.00

--------------------------------------------------


In [75]:
RF_Regressor = RandomForestRegressor(random_state=42)
evaluate_model(RF_Regressor,  x_train_scaled_df, y_train, x_test_scaled_df, y_test)


RandomForestRegressor(random_state=42)

Training Time: 99.42 seconds

Test Mean Squared Error  -- 0.00
Train Mean Squared Error  -- 0.00

Test Root Mean Squared Error  -- 0.01
Train Root Mean Squared Error  -- 0.00

Test R2 Score  -- 1.00
Train R2 Score  -- 1.00

Test Mean Absolute Error  -- 0.00
Train Mean Absolute Error  -- 0.00

Test Median Absolute Error  -- 0.00
Train Median Absolute Error  -- 0.00

Test Explained Var  -- 1.00
Train Explained Var  -- 1.00

--------------------------------------------------


In [76]:
ADA_Regressor = AdaBoostRegressor(random_state=42)
evaluate_model(ADA_Regressor, x_train_scaled_df, y_train, x_test_scaled_df, y_test)


AdaBoostRegressor(random_state=42)

